In [3]:
import matplotlib.pyplot as plt
import numpy as np ,pandas as pd
from collections import OrderedDict,Counter
import os
import statsmodels.stats.multitest as multi
import scipy.stats as stats
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
import math

In [4]:
dir="E:\\project\\singcells\\1_diff_exper_analysis\\differ-analyse\\DE_tcga_fdr005_regulon_sta\\"
dir1="E:\\project\\singcells\\6_survival\\immreg_cancer_survival_infor\\"
dir2="E:\\project\\singcells\\6_survival\\"

In [5]:
df_same=pd.read_csv(dir+"res_with_TCGA_same.txt",header=0,sep="\t")

In [6]:
df_same.head(3)

,gene,cancer,cell,State_scRNA,State_TCGA
0,RBMS1,BC,B_cell,sigDw,sigDw
1,RBM4,BC,B_cell,sigUp,sigUp
2,RC3H1,BC,Cancer,sigDw,sigDw


In [ ]:
sy_ensg=pd.read_csv("E:\\project\\geneID\\mRNA\\mRNA_unique.txt",header=None,sep="\t",index_col=0)

In [ ]:
df_same.head(3)
sy_ensg.head(3)

In [ ]:
brca=pd.read_csv(dir1+"BRCA_all.txt",header=0,index_col=0,sep="\t")
coad=pd.read_csv(dir1+"COAD_all.txt",header=0,index_col=0,sep="\t")
luad=pd.read_csv(dir1+"LUAD_all.txt",header=0,index_col=0,sep="\t")
lusc=pd.read_csv(dir1+"LUSC_all.txt",header=0,index_col=0,sep="\t")


In [ ]:
brca.head(3)

In [ ]:
df_same.head(3)

In [ ]:
gene_name=[]
cancer=[]
cell=[]
state=[]
logrank_p=[]
sig=[]

In [ ]:
for ind,cols in df_same.iterrows():
    
    g0=cols[0]
    #gene_name.append(g0)
    if cols[1]=="BC":
        gene_name.append(g0)
        cancer.append("BRCA")
        cell.append(cols[2])
        ensg=sy_ensg.loc[g0][1].split(".")[0]
        risk=brca.loc[ensg]["risk_low"]
        p=brca.loc[ensg]["p_value"]

        if risk < 0:
            state.append("risk")
        else:
            state.append("protect")

        logrank_p.append(p)
        if p<0.05:
            sig.append(1)
        else:
            sig.append(0)
    elif cols[1]=="CRC":
        gene_name.append(g0)
        cancer.append("COAD")
        cell.append(cols[2])
        ensg=sy_ensg.loc[g0][1].split(".")[0]
        risk=coad.loc[ensg]["risk_low"]
        p=coad.loc[ensg]["p_value"]

        if risk < 0:
            state.append("risk")
        else:
            state.append("protect")

        logrank_p.append(p)
        if p<0.05:
            sig.append(1)
        else:
            sig.append(0)
    elif cols[1]=="LC":
        gene_name.append(g0)
        cancer.append("LUAD,LUSC")
        cell.append(cols[2])
        ensg=sy_ensg.loc[g0][1].split(".")[0]
        risk=luad.loc[ensg]["risk_low"]
        p=luad.loc[ensg]["p_value"]

        if risk < 0:
            state1="risk"
        else:
            state1="protect"

        if p<0.05:
            sig1="1"
        else:
            sig1="0"
                
        ensg=sy_ensg.loc[g0][1].split(".")[0]
        risk1=lusc.loc[ensg]["risk_low"]
        p1=lusc.loc[ensg]["p_value"]

        if risk1 < 0:
            state2="risk"
        else:
            state2="protect"

        state.append(state1+","+state2)
        logrank_p.append(str(p)+","+str(p1))
        if p1<0.05:
            sig2="1"
        else:
            sig2="0"
        sig.append(sig1+","+sig2)
        
            

In [ ]:
df_data=pd.DataFrame()

df_data["rbp"]=gene_name
df_data["cancer"]=cancer
df_data["cell"]=cell
df_data["state"]=state
df_data["logrank_p"]=logrank_p
df_data["sig_p0.05"]=sig

In [ ]:
df_data.head(3)
df_data.shape

In [ ]:

df_data.to_csv(dir2+"tcga_desingle_same_survival_infor\\tcga_desingle_same_immreg_survival.txt",header=True,index=False,sep="\t")

In [ ]:
df_data[(df_data["sig_p0.05"]==1)|(df_data["sig_p0.05"]=="1,0")|(df_data["sig_p0.05"]=="0,1")|(df_data["sig_p0.05"]=="1,1")].to_csv(dir2+"tcga_desingle_same_survival_infor\\tcga_desingle_same_immreg_survival_sig.txt",header=True,index=False,sep="\t")